# Read the documentation : 4 short lines ;-)

**1.** Press **CTRL+F9** or click in the menu on **"Runtime"** > **"Run all"** to execute the notebook

**2.** **Edit the line below** to indicate the countries you would like to analyze

In [0]:
#countries = ['Senegal', 'Morocco', 'Belgium', 'Spain', 'France', 'Italy', 'United_States_of_America']
#countries = ['Belgium', 'Italy']

**3.** Scroll down until you see the charts

**4.** Share the URL with anyone : https://bit.ly/2ylpkeO

# Display charts about the geographic distribution of COVID-19 cases worldwide

> Data source is "today’s data on the geographic distribution of COVID-19 cases worldwide": http://bit.ly/2U9YcGS

> ECDC Dashboard : http://bit.ly/38Ysgum

In [74]:
# setup the python program
import pandas as pd
import numpy as np
from plotly.offline import iplot
import plotly.graph_objs as go
from datetime import datetime, timedelta
import locale
import urllib.request, urllib.error
import ipywidgets as widgets
import plotly.express as px
locale.setlocale(locale.LC_ALL, '')

'en_US.UTF-8'

In [75]:
import plotly.express as px
df_gapminder = px.data.gapminder()
df_gapminder.columns
df_gapminder.head(3)

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4


# Animations with plotly using gapminder data

In [0]:
#https://plotly.com/python/animations/

def display_animation(
    df,
    x="gdpPercap",
    y="lifeExp",
    animation_frame="year",
    animation_group="country",
    size="pop",
    color="continent",
    hover_name="country",
    log_x=True
    ):
  # df = px.data.gapminder()
  px.scatter(
      df, x=x, y=y, animation_frame=animation_frame, animation_group=animation_group,
      size=size, color=color, hover_name=hover_name,
      log_x=log_x
      )

# Function displaying the bar charts

In [0]:
def display_bar_chart(
    df,
    countries, 
    bar_name="New confirmed cases in", 
    bar_x='dateRep', 
    bar_y='cases',
    layout_x_title='Date', 
    layout_y_title='New confirmed cases',
    orientation='v',
    ratio_population=False
    ):
  # use the DataFrame columns for generating data

  data = []
  prefix = 'prefix_'
  for country in countries:
    # print(f"{country}")
    data_per_million = 1
    df_country = df[df['countriesAndTerritories']==country].copy(deep=False)
    #print(f'{country}, {df_country.head(3)}')
    if ratio_population == True:
      data_per_million = df_country['popData2018'].iloc[0]/1000000
    
    #print(f'data per million : {country}, {data_per_million}')
    df_country[prefix + bar_y] = df_country[bar_y].div(data_per_million)
    #print(f'{bar_x}, {bar_y}, {prefix}{bar_y}')
    #print(df_country[bar_x, bar_y, f'{prefix}{bar_y}'].head(3))
    #print(df_country.head(3))
    
    data.append(
        go.Bar(
          name=f"{bar_name} {country}",
          x=df_country[bar_x],
          y=df_country[prefix + bar_y],
          orientation=orientation,
          
        )
      )
  # plot titles and axis labels
  layout = go.Layout(
      title=f'{bar_name} {countries}',
      xaxis=dict(
          title=layout_x_title
      ),
      yaxis=dict(
          title=layout_y_title
      ),
      #legend=dict(x=0.029, y=-1.038, font_size=10),
  )

  # plot the Figure object using the 'iplot' method
  fig = go.Figure(data=data, layout=layout)
  iplot(fig)

# Load the LATEST data from the European Center for Disease Prevention and Control (ECDC)

In [0]:
def check_if_data_source_exists(url):
  """
  check if an url exists
  """
  url_exists = False
  try:
    conn = urllib.request.urlopen(url)
  except urllib.error.HTTPError as e:
      # Return code error (e.g. 404, 501, ...)
      # ...
      # print('HTTPError: {}, the file does not exist'.format(e.code))
      url_exists = False
  except urllib.error.URLError as e:
      # Not an HTTP-specific error (e.g. connection refused)
      # ...
      print('URLError: {}'.format(e.reason))
      url_exists = False
  else:
      # 200
      # ...
      print(f'Data source is : {url}')
      url_exists = True
  return (url_exists), url

In [0]:
def find_latest_datasource(url):
  """
  find the latest datasource starting today from the ECDC website

  format of the url: 
    https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-2020-03-17.xlsx
    https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-%latest_datasource_date%.xlsx

  """
  is_found=False
  i=-1
  while is_found == False and i<5:
    i += 1
    latest_datasource_date = (datetime.now() - timedelta(i)).strftime('%Y-%m-%d')
    datasource_url = url.replace('%latest_datasource_date%', latest_datasource_date)
    # print(f'Search for this latest date: {latest_datasource_date}',
    #      f'datasource_url: {datasource_url}')
    is_found, existing_datasource_date = check_if_data_source_exists(datasource_url + ".xls")
    if is_found == False:
      is_found, existing_datasource_date = check_if_data_source_exists(datasource_url + ".xlsx")

  if is_found == True:
    print(f'Latest date is : {latest_datasource_date}')
    return existing_datasource_date
  else:
    return None

In [80]:
# load the data
datasource_url_template = "https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-%latest_datasource_date%"
if datasource_url_template is None:
  print(f'There is no datasource on the ECDC website. is https://www.ecdc.europa.eu down?')
  exit
datasource_url = find_latest_datasource(datasource_url_template)

Data source is : https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-2020-04-16.xlsx
Latest date is : 2020-04-16


In [0]:
# read the data from the excel sheet
df = pd.read_excel(datasource_url)

In [82]:
# display the columns in the dataset
df.columns

Index(['dateRep', 'day', 'month', 'year', 'cases', 'deaths',
       'countriesAndTerritories', 'geoId', 'countryterritoryCode',
       'popData2018'],
      dtype='object')

In [83]:
df.shape

(11152, 10)

In [0]:
# add all countries in the table
countries = df['countriesAndTerritories'].unique()

# Display an animation
copycat of http://coronaviruswatch.ircai.org/?country=All&dashboard=news

In [85]:
# assign a continent to each country
df_country_continent_codes_list = pd.read_csv(
    'https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv',
    sep=',', 
    usecols=['alpha-3', 'region'],
    #dtype={"region": str, "alpha-3": str},
    #dtype=str
    index_col='alpha-3'
    )
print(df_country_continent_codes_list.shape)
print(df_country_continent_codes_list.columns)
#print(df_country_continent_codes_list.set_index("alpha-3", inplace=True))
# dropna
df_country_continent_codes_list=df_country_continent_codes_list.dropna()
print(df_country_continent_codes_list.head(5))
df_country_continent_codes_list['region'] = df_country_continent_codes_list['region'].astype(str)
df_country_continent_codes_list['region']
df_country_continent_codes_list.dtypes
#df_country_continent_codes_list = df_country_continent_codes_list.apply(pd.to_pickle)

(249, 1)
Index(['region'], dtype='object')
          region
alpha-3         
AFG         Asia
ALA       Europe
ALB       Europe
DZA       Africa
ASM      Oceania


region    object
dtype: object

In [0]:
## create a dictionnary including alpha3:continent
mymap = df_country_continent_codes_list['region'].to_dict()
#mymap

In [0]:
df['continent'] = df['countryterritoryCode'].map(mymap)

In [0]:
#growth last 7 days for all countries
df_all = pd.DataFrame(columns=df.columns)
periods = 1 #moving average
for country in countries:
  # country = "Afghanistan"
  df_growth = df[df["countriesAndTerritories"]==country].copy(deep=True)
  #df_growth['dateRep'] = df_growth["dateRep"].astype(str)
  #df_growth['dateRep'] = df_growth['dateRep'].dt.date
  df_growth = df_growth.sort_values(by=["countriesAndTerritories", "dateRep"], ascending=True)
  df_growth['cases_sum_to_date'] = df_growth['cases'].cumsum()
  df_growth['deaths_sum_to_date'] = df_growth['deaths'].cumsum()
  df_growth['cases_growth'] = df_growth['cases_sum_to_date'].pct_change(periods=periods, fill_method='pad')
  df_growth['deaths_growth'] = df_growth['deaths_sum_to_date'].pct_change(periods=periods, fill_method='pad')
  df_all = df_all.append(df_growth,ignore_index=True)
  # print(f"{country} : {df_growth.shape}")

df_all["deaths"] = df_all["deaths"].astype(str).astype(int)
df_all["cases"] = df_all["cases"].astype(str).astype(int)
df_all['dateRep'] = df_all["dateRep"].astype(str)
df_all = df_all.sort_values(by=["dateRep"], ascending=True)
df_all = df_all.dropna()

In [89]:
df_all.dtypes

dateRep                     object
day                         object
month                       object
year                        object
cases                        int64
deaths                       int64
countriesAndTerritories     object
geoId                       object
countryterritoryCode        object
popData2018                float64
continent                   object
cases_sum_to_date          float64
deaths_sum_to_date         float64
cases_growth               float64
deaths_growth              float64
dtype: object

In [90]:
df_all['continent'].unique()

array(['Asia', 'Europe', 'Oceania', 'Americas', 'Africa'], dtype=object)

In [112]:
df_all.head(1)

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018,continent,cases_sum_to_date,deaths_sum_to_date,cases_growth,deaths_growth
2184,2020-01-11,11,1,2020,0,1,China,CN,CHN,1.392730e+09,Asia,59.0,1.0,0.0,inf


In [92]:
df_all.describe()

,cases,deaths,popData2018,cases_sum_to_date,deaths_sum_to_date,cases_growth,deaths_growth
count,4171.000000,4171.000000,4.171000e+03,4171.000000,4171.000000,4171.000000,4171.000000
mean,481.009830,32.678255,7.878099e+07,7829.266842,423.494126,inf,inf
std,2194.555682,165.694019,2.421809e+08,33365.275563,2029.650424,NaN,NaN
min,0.000000,0.000000,3.145800e+04,1.000000,1.000000,0.000000,0.000000
25%,5.000000,0.000000,4.176873e+06,90.000000,2.000000,0.026682,0.000000
50%,35.000000,1.000000,1.156520e+07,467.000000,7.000000,0.080825,0.048780
75%,167.000000,5.000000,4.964868e+07,2206.500000,46.000000,0.165218,0.200334
max,35527.000000,4928.000000,1.392730e+09,639664.000000,30985.000000,inf,inf


In [127]:
# df_all[(df_all['continent']=="Europe") | (df_all['continent']=="Asia")]
continent = "Europe"
df_to_plot = df_all.copy(deep=True)
df_to_plot = df_to_plot.replace([np.inf, -np.inf], np.nan)
f_to_plot = df_to_plot.dropna()
df_to_plot = df_to_plot[(df_to_plot['continent']==continent)]
px.scatter(df_to_plot, 
           x="cases_sum_to_date", 
           y="cases_growth", 
           animation_frame="dateRep", 
           animation_group="countriesAndTerritories",
           size="deaths_sum_to_date", 
           color="continent", 
           hover_name="countriesAndTerritories",
           log_x=False, 
           size_max=50, 
           range_x=[df_to_plot['cases_sum_to_date'].min(),
                    df_to_plot['cases_sum_to_date'].max()], 
           range_y=[df_to_plot['cases_growth'].min(),
                    df_to_plot['cases_growth'].quantile(0.75)]
          )
# df_all['cases_sum_to_date'].max()

In [128]:
px.bar(df_to_plot, 
           x="continent", 
           y="deaths_sum_to_date", 
           animation_frame="dateRep", 
           animation_group="countriesAndTerritories",
           #size="deaths_sum_to_date", 
           color="continent", 
           hover_name="countriesAndTerritories",
           log_x=False, 
           #size_max=50, 
           range_y=[df_to_plot['cases_sum_to_date'].min(),
                    df_to_plot['cases_sum_to_date'].max()], 
           #range_y=[df_to_plot['cases_growth'].min(),
           #         df_to_plot['cases_growth'].quantile(0.75)]
          )

In [0]:
df_gapminder = px.data.gapminder()

In [122]:
df_gapminder.describe()

,year,lifeExp,pop,gdpPercap,iso_num
count,1704.00000,1704.000000,1.704000e+03,1704.000000,1704.000000
mean,1979.50000,59.474439,2.960121e+07,7215.327081,425.880282
std,17.26533,12.917107,1.061579e+08,9857.454543,248.305709
min,1952.00000,23.599000,6.001100e+04,241.165877,4.000000
25%,1965.75000,48.198000,2.793664e+06,1202.060309,208.000000
50%,1979.50000,60.712500,7.023596e+06,3531.846989,410.000000
75%,1993.25000,70.845500,1.958522e+07,9325.462346,638.000000
max,2007.00000,82.603000,1.318683e+09,113523.132900,894.000000


In [94]:
df_to_plot['cases_growth'].describe()

count    847.000000
mean       0.152652
std        0.334323
min        0.000000
25%        0.028717
50%        0.090164
75%        0.181099
max        6.000000
Name: cases_growth, dtype: float64

In [95]:
df_to_plot['cases_growth'].quantile(0.75)

0.1810989643268125

In [96]:
df_to_plot = df_all.copy(deep=True)
df_to_plot = df_to_plot.replace([np.inf, -np.inf], np.nan)
df_to_plot = df_to_plot.dropna()
df_to_plot.describe()

,cases,deaths,popData2018,cases_sum_to_date,deaths_sum_to_date,cases_growth,deaths_growth
count,4010.000000,4010.000000,4.010000e+03,4010.000000,4010.000000,4010.000000,4010.000000
mean,499.217207,33.940399,8.015261e+07,8137.266833,440.447382,0.126689,0.159736
std,2236.249807,168.866156,2.448871e+08,33992.480074,2068.204509,0.204027,0.331486
min,0.000000,0.000000,3.145800e+04,1.000000,1.000000,0.000000,0.000000
25%,6.000000,0.000000,4.176873e+06,104.000000,2.000000,0.025890,0.000000
50%,38.000000,1.000000,1.443902e+07,514.000000,8.000000,0.078474,0.039578
75%,181.000000,5.000000,5.139301e+07,2415.000000,52.000000,0.158121,0.181818
max,35527.000000,4928.000000,1.392730e+09,639664.000000,30985.000000,5.000000,4.600000


In [97]:
df_to_plot.describe()

,cases,deaths,popData2018,cases_sum_to_date,deaths_sum_to_date,cases_growth,deaths_growth
count,4010.000000,4010.000000,4.010000e+03,4010.000000,4010.000000,4010.000000,4010.000000
mean,499.217207,33.940399,8.015261e+07,8137.266833,440.447382,0.126689,0.159736
std,2236.249807,168.866156,2.448871e+08,33992.480074,2068.204509,0.204027,0.331486
min,0.000000,0.000000,3.145800e+04,1.000000,1.000000,0.000000,0.000000
25%,6.000000,0.000000,4.176873e+06,104.000000,2.000000,0.025890,0.000000
50%,38.000000,1.000000,1.443902e+07,514.000000,8.000000,0.078474,0.039578
75%,181.000000,5.000000,5.139301e+07,2415.000000,52.000000,0.158121,0.181818
max,35527.000000,4928.000000,1.392730e+09,639664.000000,30985.000000,5.000000,4.600000


In [98]:
df_all[df_all['continent']==continent]['cases_growth']

10817    0.045455
10818    0.289855
10819    0.157303
10820    0.213592
10821    0.272000
           ...   
9274     0.096154
1950     0.048732
6792     0.082978
2069     0.111111
10984    0.020725
Name: cases_growth, Length: 847, dtype: float64

In [99]:
px.scatter(df_gapminder, x="gdpPercap", y="lifeExp", animation_frame="year", animation_group="country",
           size="pop", color="continent", hover_name="country",
           log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])

# What is the range of data that has been collected from the ECDC?

In [100]:
start_date = df.dateRep.min().strftime("%b %d %Y")
end_date = df.dateRep.max().strftime("%b %d %Y")

print(
    f'DATE RANGE:\n',
    f'start date : {start_date}\n', 
    f'end date : {end_date}'
      )

DATE RANGE:
 start date : Dec 31 2019
 end date : Apr 16 2020


# Display Totals : Confirmed   and Deaths

In [101]:
cv19_countries_day = df.groupby(by=['dateRep','countriesAndTerritories']).sum()[['cases','deaths']]
Total_confirmed = cv19_countries_day.groupby('dateRep').sum()[['cases','deaths']].sum()['cases']
Total_deaths = cv19_countries_day.groupby('dateRep').sum()[['cases','deaths']].sum()['deaths']
print(f"{end_date}:\n Total Confirmed cases: {Total_confirmed:n}\n Total deaths: {Total_deaths:n}")

Apr 16 2020:
 Total Confirmed cases: 2,029,930
 Total deaths: 136,320


# Group the data by daily-New Confirmed cases (WW)

In [0]:
cv19_countries_day_new_confirmed_cases = df.groupby(by=['dateRep'], as_index=False)['cases'].sum()
# print(f'WW New Confirmed cases first-5 records:\n\n {cv19_countries_day_new_confirmed_cases.head(5)}')
# print(f'\nWW New Confirmed cases last-5 records:\n\n {cv19_countries_day_new_confirmed_cases.tail(5)}')

# Group the data by daily Deaths (WW)

In [0]:
cv19_countries_day_new_deaths = df.groupby(by=['dateRep'], as_index=False)['deaths'].sum()
# print(f'WW New deaths first-5 records:\n\n {cv19_countries_day_new_deaths.head(5)}')
# print(f'nWW New deaths last-5 records:\n\n {cv19_countries_day_new_deaths.tail(5)}')

# Draw a chart with daily New (WW) Confirmed Cases and Deaths

In [104]:
data = [
        go.Scatter(
        name="WW New Confirmed cases",
        x=cv19_countries_day_new_confirmed_cases['dateRep'],
        y=cv19_countries_day_new_confirmed_cases['cases']
        ),
        go.Scatter(
        name="WW New deaths",
        x=cv19_countries_day_new_deaths['dateRep'],
        y=cv19_countries_day_new_deaths['deaths']
        ),
]

# plot titles and axis labels
layout = go.Layout(
    title=f'Worldwide New Confirmed cases and deaths',
    xaxis=dict(
        title='Date'
    ),
    yaxis=dict(
        title='New Confirmed cases/New deaths'
    )
)

# plot the Figure object using the 'iplot' method
fig = go.Figure(data=data, layout=layout)
iplot(fig)


# ACTION : Choose the countries you want to compare or update the list of countries you want to study (Multi-selection with CTRL key)

countries_widget = widgets.SelectMultiple(
    options=df['countriesAndTerritories'].unique().tolist(),
    value=countries,
    rows=10,
    description='countriesAndTerritories',
    disabled=False
)

try:
  display(countries_widget)
  #print('you are running a jupyter notebook')
except:
  print('you are not running a jupyter notebook')

  if countries_widget is not None:
    if countries_widget.value is not None:
      countries = countries_widget.value
      
print(f'Selected countries : {countries}')

# Draw a chart including New Confirmed Cases in Italy and Belgium

## Some hints regarding the dynamic charts

> functions are available on the top-left of the screen. Click on the buttons

> the user can zoom in/zoom out

> click on a legend e.g., new confirmed cases (on the right) and focus on one single dataset

> click-and-drag your mouse to zoom on a dedicated area

> double click on the chart to reset the zoom

In [105]:
# display a bart chart for confirmed cases
display_bar_chart(
    df,
    countries, 
    bar_name="New confirmed cases in", 
    bar_x='dateRep', 
    bar_y='cases',
    layout_x_title='Date', 
    layout_y_title='New confirmed cases',
    ratio_population=False,
    )
# display a bart chart for new deceased people
display_bar_chart(
    df,
    countries,
    bar_name="New deaths in", 
    bar_x='dateRep', 
    bar_y='deaths',
    layout_x_title='Date', 
    layout_y_title='New deaths',
    ratio_population=False,
    )

# display a bart chart for new deceased people
display_bar_chart(
    df,
    countries,
    bar_name="New deaths per Mio in", 
    bar_x='dateRep', 
    bar_y='deaths',
    layout_x_title='Date', 
    layout_y_title='New deaths',
    ratio_population=True,
    )

In [106]:
df['popData2018']
df[df['countriesAndTerritories']=='Belgium']['popData2018'].iloc[0]/1000000

11.422068

# Are they Correlations between the fields 'deaths' and 'New cases'?

In [0]:
def display_correlation(df, selected_citizenship= 'Belgian', method='pearson'):
  correlation = df['cases'].corr(df['deaths'], method=method)
  print(f"correlation using '{method}'' method between '{selected_citizenship}' 'New cases' and 'deaths': {correlation}")

In [108]:
for country in countries:
  display_correlation(df[df['countriesAndTerritories']==country], country, method='pearson')


correlation using 'pearson'' method between 'Afghanistan' 'New cases' and 'deaths': 0.6640678105429902
correlation using 'pearson'' method between 'Albania' 'New cases' and 'deaths': 0.38059055651756774
correlation using 'pearson'' method between 'Algeria' 'New cases' and 'deaths': 0.8507305009981813
correlation using 'pearson'' method between 'Andorra' 'New cases' and 'deaths': 0.1415288622448368
correlation using 'pearson'' method between 'Angola' 'New cases' and 'deaths': 0.41486590682143165
correlation using 'pearson'' method between 'Anguilla' 'New cases' and 'deaths': nan
correlation using 'pearson'' method between 'Antigua_and_Barbuda' 'New cases' and 'deaths': 0.39811167621296767
correlation using 'pearson'' method between 'Argentina' 'New cases' and 'deaths': 0.682776477684132
correlation using 'pearson'' method between 'Armenia' 'New cases' and 'deaths': 0.6056567785413942
correlation using 'pearson'' method between 'Aruba' 'New cases' and 'deaths': -0.12138522730031968
corre

# CHANGELOG

v.0.0.7

> rename names due to changes in the raw data by the ECDC

v.0.0.2

> fix: ECDC has changed the format of the file

  > filename is renamed *.xlsx into *.xls

> feat: add widget to select the countries you want a analysis about

> refactor: one function displays diverse bart charts

v.0.0.1 
  
> fix: ECDC has changed the format of the file

    > filename is renamed *.xls into *.xlsx
  
    > renamed fields: CountryExp -> Countries and territories, NewConfCases -> Cases, Newdeaths -> Deaths
  
    > removed fields: Gaul1Nuts1, EU

v.0.0.0 

    > feat: display charts for the World, Belgium and Italy

# LICENSE

(c) 2010-2020 Abdelkrim Boujraf, alt-f1 sprl <http://www.alt-f1.be>, Abdelkrim Boujraf <http://www.alt-f1.be/literature.html>

This work is licensed under a Creative Commons Attribution 4.0 International License: <http://creativecommons.org/licenses/by/4.0>

* Share — copy and redistribute the material in any medium or format 
* Adapt — remix, transform, and build upon the material for any purpose, even commercially

Vous êtes autorisé à <https://creativecommons.org/licenses/by/4.0/deed.fr>: 

* Partager : copier, distribuer et communiquer le matériel par tous moyens et sous tous formats
* Adapter : remixer, transformer et créer à partir du matériel pour toute utilisation, y compris commerciale